# Software de control y gestión de datos

````{admonition} Resumen 
:class: tip

Este informe presenta los trabajos relativos al sistema de control y gestión de datos implementado en FLATCLASS. La plataforma utiliza un conjunto de microservicios para gestionar de forma coordinada la adquisición de imágenes, el procesamiento de datos, la clasificación automática y la supervisión. La modularidad de la arquitectura desacopla los subsistemas de visión artificial, análisis estadístico, interfaz de usuario y comunicaciones, proporcionando escalabilidad, tolerancia a fallos y facilidad de extensión, manteniendo la estabilidad en entornos productivos.

**Entregable**: E5.1  
**Versión**: 1.0  
**Autor**: Javier Álvarez Osuna  
**Email**: javier.osuna@fishfarmfeeder.com  
**ORCID**: [0000-0001-7063-1279](https://orcid.org/0000-0001-7063-1279)  
**Licencia**: CC-BY-4.0  
**Código proyecto**: IG408M.2025.000.000072

```{figure} .././assets/FLATCLASS_logo_publicidad.png
:width: 100%
:align: center
```
````

## Introducción

El sistema FLATCLASS se sustenta sobre una arquitectura software distribuida diseñada específicamente para operar en tiempo real bajo las exigencias de un entorno industrial acuícola. La gestión de datos y el control del flujo operativo se realizan mediante una combinación de microservicios especializados que coordinan la adquisición de imágenes, el procesamiento morfométrico, la clasificación automática y la supervisión del proceso. Esta arquitectura modular permite desacoplar los distintos componentes —visión artificial, motor estadístico, interfaz de usuario y sistema de comunicaciones— asegurando escalabilidad, resiliencia frente a fallos y capacidad de ampliar funcionalidades sin comprometer la estabilidad del sistema en producción.

Para garantizar una comunicación eficiente entre módulos, FLATCLASS emplea tecnologías orientadas a sistemas IoT industriales. El intercambio de datos en tiempo real se realiza mediante MQTT, un protocolo de mensajería ligera optimizado para latencias bajas y tolerancia a desconexiones puntuales, facilitando la transmisión continua de mediciones morfométricas desde la visión artificial hacia el backend central. Este backend está desarrollado en Node.js, emplea Prisma como ORM para garantizar integridad y tipado estricto en los accesos a la base de datos, y almacena la información en MongoDB, elegido por su capacidad de gestionar series temporales y formatos documentales de manera flexible. El despliegue de los diferentes módulos se realiza mediante Docker, lo que permite asegurar entornos homogéneos tanto en el Edge (CPU-GPU de la máquina) como en servidores externos de supervisión y análisis.


El sistema FLATCLASS se sustenta sobre una arquitectura de control distribuida y modular, diseñada específicamente para operar en tiempo real bajo las exigentes condiciones de un entorno industrial acuícola. Esta arquitectura desacopla funcionalmente los componentes críticos —visión artificial, lógica de negocio, interfaz de usuario y sistemas mecatrónicos— garantizando escalabilidad, resiliencia frente a fallos y una alta capacidad de mantenimiento.

El corazón del **sistema de control** reside en Node-RED, que actúa como orquestador principal de los flujos de datos y comandos. Su entorno de programación visual basado en flujos ha permitido implementar de manera ágil y robusta la lógica de secuenciación del proceso de clasificación. Node-RED gestiona de forma nativa los protocolos de comunicación industrial empleados:

- OPC-UA: Se utiliza para la comunicación con el sistema de visión artificial y los actuadores mecatrónicos de clasificación, aprovechando su modelo de información estandarizado, seguridad integrada y capacidad para operar en entornos industriales confiables. A través de OPC-UA, se reciben en tiempo real los resultados del análisis morfométrico (longitud, peso estimado, etc.) y se envían los comandos de actuación a los ejectores o mecanismos de separación.

- MQTT: Este protocolo de mensajería pub/sub ligero se emplea para la comunicación con sensores adicionales, microservicios periféricos y para la transmisión de datos de estado y telemetría. Su ligereza y tolerancia a interrupciones de red lo hacen ideal para conectar dispositivos IoT dentro de la granja.

La supervisión y control por parte de los operarios se realiza a través de una **Interfaz de Usuario (HMI)** moderna y reactiva, desarrollada en React. Esta interfaz proporciona una visualización en tiempo real del estado del proceso, métricas de rendimiento, historiales de clasificación y permite la intervención manual cuando es requerido.

Para la **persistencia de datos**, el sistema utiliza MongoDB como base de datos principal, seleccionada por su flexibilidad esquemática para adaptarse a la evolución del proyecto y su eficiencia en el manejo de datos de tipo temporal y eventos discretos. El acceso a la base de datos no se realiza directamente, sino a través de Prisma como capa de ORM (Object-Relational Mapping). Prisma garantiza un acceso a los datos tipado, seguro y eficiente, previniendo inyecciones de código y asegurando la integridad de las operaciones entre la aplicación React/Node.js y MongoDB.

La lógica de negocio específica (por ejemplo, algoritmos de cálculo, gestión de lotes, etc.) se implementa en microservicios especializados, desarrollados principalmente en Node.js. Esta modularidad permite que cada componente pueda ser desarrollado, desplegado y escalado de manera independiente. El despliegue unificado de todos los componentes —Node-RED, los microservicios, la base de datos y la interfaz de React— se gestiona mediante Docker. La contenerización asegura la consistencia del entorno de ejecución entre desarrollo, staging y producción, simplifica el despliegue y favorece la filosofía de DevOps.

Sobre esta infraestructura se superponen los dos módulos funcionales principales: **Muestreo**, que construye el modelo estadístico del lote mediante análisis de distribuciones, y **Clasificación**, que aplica en tiempo real dichos criterios a cada pez detectado, permitiendo dividir los individuos en categorías morfométricas basadas en criterios objetivos y reproducibles.

En conclusión, la arquitectura de FLATCLASS combina tecnologías punteras de IoT industrial (Node-RED, OPC-UA, MQTT) con un stack web moderno (React, Node.js, Prisma) y una base de datos NoSQL flexible (MongoDB), todo ello contenerizado con Docker. Esta sinergia proporciona una plataforma de control robusta, escalable y preparada para la integración futura de nuevos módulos de análisis y automatización.


## Módulo de muestreo

El módulo de muestreo permite establecer, antes de cada sesión productiva, los límites operativos que definirán qué individuos serán considerados pequeños, medianos y grandes en función de su superficie real obtenida por el sistema de visión artificial. Esta calibración es fundamental porque garantiza que el proceso de clasificación actúe conforme al estado real del lote, evitando desviaciones debidas a variaciones biológicas, ópticas o de crecimiento. En el contexto FLATCLASS, este módulo constituye el nexo entre la observación objetiva generada por el módulo de visión artificial y la decisión operativa que el sistema aplicará posteriormente en el proceso de sorting o clasificación.

Desde la perspectiva del usuario, el módulo presenta un flujo guiado pensado para operarios de hatchery, técnicos en acuicultura y responsables de producción. En el estado inicial, la interfaz muestra la opción de seleccionar tanque de origen y un botón “Start”, indicando al operador que la calibración se iniciará mediante la captura en tiempo real de medidas de superficie enviadas por el sistema de visión artificial. Una vez iniciada, el sistema entra en modo “calibrating” y comienza a mostrar el número de peces procesados, que crece dinámicamente conforme pasan individuos por el canal de medición. Esta retroalimentación visual permite al usuario supervisar el progreso en tiempo real.

Durante el proceso, los valores de superficie se almacenan temporalmente en memoria y se someten a un cálculo dinámico que genera una distribución estadística. Completada la fase de captura, el módulo pasa a la visualización de resultados, donde se representa un histograma agrupado en intervalos (bins), acompañado por una curva teórica ajustada a una distribución normal. El usuario dispone entonces de campos donde introducir valores mínimos y máximos que definirán los rangos de tamaño, y el sistema recalcula las distribuciones correspondientes, mostrando los porcentajes de individuos en cada categoría. Finalmente, el operador puede guardar (“Save”), descartar (“Discard”) o ajustar valores mediante “Recalculate”, lo que confiere flexibilidad y comprensión completa del proceso.

En las siguientes figuras se recogen algunas de las pantallas correspondientes a la interfaz de usuario que definen la operativa anteriormente descrita.

```{figure} .././assets/Calibracion.png
:name: WP5_1
:alt: Soft_Modulo_calibracion
:align: center

Detalle de la interfaz de usuario de muestreo
```


 

### Especificación formal de eventos

```{figure} .././assets/BPMN-Calibracion.png
:name: WP5_2
:alt: BPMN_calibracion
:width: 70%
:align: center

Diagrama de eventos del módulo de muestreo
```

El funcionamiento interno del módulo está regido por un conjunto de eventos formales que garantizan un comportamiento determinista y trazable. Al iniciar, el sistema se encuentra en el estado `IDLE`, y sólo el evento `START_CALIBRATION` puede transicionar hacia `CALIBRATING`. Este evento se acompaña de un payload transmitido por MQTT al backend, donde se activa la recogida de datos provenientes del microservicio de visión artificial. Durante el estado CALIBRATING se procesan eventos `NEW_MEASUREMENT({long, width, surface)`, los cuales actualizan el contador de peces y alimentan el vector base de datos estadísticos. Si el operador pulsa `PAUSE`, se genera el evento `CALIBRATION_PAUSED`, manteniendo los datos en memoria pero deteniendo la adquisición. Un evento complementario RESUME revierte el estado a CALIBRATING.

El evento `STOP_CALIBRATION` finaliza la captura y activa `CALCULATION_READY`, enviando al frontend todos los valores agregados que permiten generar la distribución. Posteriormente, los eventos `RECALCULATE`, `SAVE_CALIBRATION` y `DISCARD_CALIBRATION` gestionan, respectivamente, la modificación de rangos, la persistencia de datos en MongoDB y el reinicio total del módulo sin almacenamientos permanentes. El backend supervisor registra cada evento en un log técnico para auditoría, garantizando trazabilidad, control de errores y sincronización continua entre software, visión artificial y la máquina física.


### Fundamento matemático

El análisis estadístico aplicado en el módulo de Calibración del sistema FLATCLASS se basa en la caracterización cuantitativa de la distribución de superficies morfométricas obtenidas durante la fase de muestreo. Dado un conjunto de mediciones de superficie real del pez, denotado como  
$$x_1, x_2, \dots, x_n,$$  
se busca describir la tendencia central, la dispersión y la forma de la distribución con el fin de establecer límites operativos para la segmentación en tallas (pequeño, mediano y grande).

#### Media aritmética

La **media** es el estimador de tendencia central que resume el valor típico del lote. Se define como:

$$
\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i
$$

donde $\bar{x}$ representa el valor medio de la superficie y $n$ es el número total de peces observados.

#### Varianza

La **varianza** cuantifica la dispersión de las mediciones respecto a la media, indicando cuán heterogéneo es el lote:

$$
\sigma^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

Una varianza baja indica que las superficies están concentradas alrededor de la media, mientras que una varianza elevada indica un lote más heterogéneo.

#### Desviación estándar

La **desviación estándar**, definida como la raíz cuadrada de la varianza, es el parámetro de dispersión más utilizado por su interpretación directa en las mismas unidades que la variable original:

$$
\sigma = \sqrt{\sigma^2}
$$

Este parámetro se utiliza en la superposición de una curva teórica sobre el histograma de mediciones.

#### Distribución normal teórica

Para proporcionar una referencia visual y ayudar en la interpretación del lote, el sistema FLATCLASS superpone una **distribución normal teórica** ajustada usando la media $\bar{x}$ y la desviación estándar $\sigma$ calculadas:

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} \, \exp\!\left( -\frac{1}{2} 
\left( \frac{x - \bar{x}}{\sigma} \right)^2 \right)
$$

Esta curva no implica que los datos sean estrictamente normales, pero permite visualizar de forma intuitiva la estructura del lote, facilitando la toma de decisiones sobre límites de clasificación.

#### Justificación estadística para la clasificación

La segmentación en rangos de talla (pequeño, mediano y grande) se fundamenta en:

- La forma unimodal que presentan habitualmente las distribuciones de superficie en alevines y juveniles.  
- La estabilidad de la superficie como descriptor morfométrico incluso con variaciones lumínicas moderadas.  
- La relación empírica entre superficie proyectada y crecimiento lineal/ponderal.

El uso de los parámetros $\bar{x}$ y $\sigma$ permite definir rangos de talla coherentes, reproducibles y basados en la estructura real del lote, reduciendo así la subjetividad en la calibración.


## Módulo de Clasificación.

El módulo de Clasificación del sistema FLATCLASS constituye el núcleo lógico que convierte las medidas morfométricas generadas por la visión artificial en decisiones operativas de clasificación de peces planos por tamaño en centros de alevinaje. Desde una perspectiva de arquitectura de sistemas, este módulo actúa como orquestador entre el subsistema de visión (cámara, algoritmos de segmentación y extracción de contornos), el modelo de estimación de peso basado en IA y los actuadores mecánicos de desvío hacia tanques destino. Su propósito principal es transformar, en tiempo real, las observaciones individuales de cada pez (longitud, anchura y superficie proyectada) en información agregada de producción (número de individuos por rango de talla, distribución porcentual y biomasa por categoría), apoyando así la toma de decisiones en procesos de clasificación y planificación de lotes, en línea con la tendencia actual hacia biometría no invasiva y sistemas de clasificación automática en acuicultura de precisión.

La lógica de Clasificación se apoya en una calibración previa del sistema de visión, que permite convertir dimensiones expresadas en píxeles en unidades métricas mediante un factor de escala obtenido a partir de patrones conocidos o parámetros geométricos del conjunto cámara-óptica-altura. Esta conversión es esencial para vincular las medidas morfométricas bidimensionales con el peso vivo de los alevines, siguiendo relaciones empíricas estudiadas en el Paquete de Trabajo 3 relativo a la Predicción del Peso (E3.1)

En el caso de FLATCLASS, el modelo IA de inferencia de peso implementado en Python permite que en el proceso de Clasificación, para cada pez, ofrecer una estimación puntual del peso coherente con las relaciones morfométricas obtenidas por el sistema de visión artificial.

Desde el punto de vista operativo, el módulo se presenta al usuario a través de una interfaz gráfica estructurada en tres paneles verticales correspondientes a las categorías **Small**, **Medium** y **Large**. Antes de iniciar cualquier ciclo de trabajo, el sistema se encuentra en un estado inicial `Idle`, con el contador total de peces a cero, los tanques de origen y destino sin asignar (valor `NULL`) y unos rangos de peso–talla asociados a cada clase derivados de la sesión de calibración. Esta pantalla inicial proporciona una visión clara del estado de preparación del sistema, anticipando qué rangos de peso se aplicarán durante la sesión de clasificación, lo que reduce la ambigüedad operativa y facilita la comunicación entre operarios y responsables de alevinaje.

El flujo funcional se inicia cuando el usuario define el tanque de origen desde el que se van a extraer los peces para su clasificación. Esta selección, accesible en el panel lateral izquierdo, vincula la sesión con un identificador físico dentro de la granja (por ejemplo, tanque `N3` o `N15`), permitiendo la trazabilidad posterior de los datos de clasificación.

El siguiente paso crítico para el usuario es la selección de los tanques de destino por categoría de tamaño. Para cada columna (Small, Medium, Large), el botón **Select destination tank** abre un diálogo de selección en el que se listan los tanques disponibles y autorizados para recibir peces de ese rango de peso. La lógica interna del módulo verifica que no existan colisiones (por ejemplo, dos categorías apuntando al mismo tanque si la estrategia de manejo no lo permite) y que los tanques seleccionados cumplen los requisitos de volumen, densidad máxima admisible y compatibilidad de tamaño. Una vez asignados los tanques, su identificador se refleja en la parte inferior de cada columna, y el estado del módulo pasa de `ready`, habilitando el botón de **Start** para lanzar la sesión.

Con la sesión preparada y el usuario pulsando **Start**, se activa el estado `Classificating`, en el que el módulo comienza a suscribirse de forma efectiva al flujo de medidas emitido por el subsistema de visión artificial a través de MQTT. Cada vez que un pez atraviesa la zona de inspección, el módulo de visión genera un mensaje con los valores de superficie proyectada en píxeles \(A_i^{px}\), longitud \(L_i^{px}\), anchura \(H_i^{px}\). Estos valores se transforman a milímetros aplicando el factor de escala \(\alpha\) (mm/px) derivado de la calibración, de modo que \(A_i = \alpha^2 A_i^{px}\), \(L_i = \alpha L_i^{px}\) y \(H_i = \alpha H_i^{px}\). Este preprocesamiento garantiza la coherencia dimensional y prepara los datos para el modelo de estimación de peso.

La estimación de peso se realiza mediante un modelo de IA desplegado como microservicio Python que implementa una función
$
f_\theta:\ (L_i,A_i)\mapsto \hat{W}_i,
$
donde $\theta$ representa el vector de parámetros entrenados a partir de datos históricos de la propia granja y de estudios previos sobre relaciones morfométricas y peso en peces planos (ver E3.1 - "Predicción del peso"). En una formulación típica, el modelo combina una transformación logarítmica y una regresión multivariante, aproximando expresiones del tipo
$$
\log \hat{W}_i = \beta_0 + \beta_1\log L_i + \beta_3\log A_i.
$$
equivalente a un modelo alométrico del tipo:

$$
\hat{W}_i = k·L_{i}^a·A_{i}^b
$$
Esta aproximación permite alcanzar coeficientes de determinación elevados cuando la calibración geométrica del sistema de visión es estable y el rango de tallas está bien representado en la muestra de entrenamiento.



```{figure} .././assets/Clasificacion.png
:name: WP5_3
:alt: Soft_Modulo_clasificacion
:align: center
:width: 70%

Detalle de la interfaz de usuario de Clasificación
```

Una vez calculado ($\hat{W}_i$), el módulo de Clasificación aplica las reglas de asignación de categoría. Sean ($\tau_1$) y ($\tau_2$) los umbrales de peso que delimitan las categorías Small, Medium y Large, definidos en la fase de calibración. La función de clasificación se define entonces como:

$$
C(\hat{W}_i) =
\begin{cases}
\text{Small}, & \hat{W}_i < \tau_1,\\
\text{Medium}, & \tau_1 \le \hat{W}_i \le \tau_2,\\
\text{Large}, & \hat{W}_i > \tau_2,
\end{cases}
$$

de modo que el resultado de $C(\hat{W}_i)$ no sólo determina el conteo estadístico, sino que se transmite a la lógica de control de la máquina física para accionar, con la temporización adecuada, la compuerta neumática o desviador correspondiente a la categoría asignada, asegurando que el pez es redirigido al tanque destino previamente configurado.

En paralelo a la asignación de categoría, el módulo mantiene una contabilidad precisa del número de peces procesados. Para cada clase $(k \in {\text{Small}, \text{Medium}, \text{Large}})$, se mantiene un contador $(N_k$ que se incrementa cada vez que un pez es clasificado en dicha categoría. El contador total de peces procesados en la sesión se define como $N_{\text{tot}} = \sum_k N_k$ y se refleja en tiempo real en el panel lateral (`Total peces`), proporcionando al usuario una medida inmediata del avance de la operación. Esta lógica está respaldada por un identificador único de pez (`fish_id`) asociado a cada medición recibida, lo que evita dobles conteos en caso de retransmisiones o duplicados en la capa de comunicación.

De forma complementaria, el módulo calcula en tiempo real la distribución porcentual de individuos por categoría, variable crítica para evaluar el resultado de la clasificación y su alineamiento con la estructura de tallas esperada en el lote. Matemáticamente, el porcentaje de una categoría $(k)$ se define como $p_k = 100 \cdot \frac{N_k}{N_{\text{tot}}}$ expresado en tanto por ciento y mostrado en la parte inferior de cada columna. Esta información permite al operario detectar rápidamente desviaciones.

### Especificación formal de eventos

```{figure} .././assets/BPMN-Clasificacion.png
:name: WP5_5
:alt: BPMN_clasificacion
:width: 70%
:align: center

Diagrama de eventos del módulo de clasificación
```

La operación del módulo de Clasificación se articula como una **máquina de eventos asincrónicos** fundamentada en la interacción entre el usuario, la interfaz React, el backend Node.js/Prisma, el broker MQTT OPC-UA, el servicio de visión artificial y el servicio de estimación de peso por IA. El ciclo comienza con una fase previa de configuración, en la cual el usuario define el tanque de origen y los tanques destino asociados a cada categoría de tamaño. Esta acción genera un evento `CONFIGURE_SESSION`, transmitido desde la interfaz al backend, que a su vez valida la consistencia de la configuración consultando la base de datos. Una vez verificada la disponibilidad de los recursos y la coherencia de los parámetros operativos, el backend emite hacia la interfaz el evento `SESSION_READY`, que habilita el inicio formal de la sesión.

El evento crítico que activa el ciclo de clasificación es `START_CLASSIFICATION`. Al recibir esta orden desde la interfaz, el backend inicializa una nueva sesión identificada mediante un `session_id` único y solicita al broker MQTT la suscripción al tópico `flatclass/vision/measurements`, que es donde el servicio de visión publicará cada nueva medición asociada a un pez detectado. Paralelamente, el backend emite un comando MQTT (`{command:"start", session_id}`) que sincroniza el arranque de todos los módulos implicados. A partir de ese momento, el flujo de eventos entra en el **bucle principal de clasificación**, donde cada pez representa una iteración independiente del ciclo.

Cada detección del sistema de visión genera un evento `NEW_MEASUREMENT`, que incluye los valores en píxeles de área, longitud, anchura. Este evento es publicado por el servicio de visión y encaminado por MQTT hasta el backend. Al recibir `NEW_MEASUREMENT`, el backend ejecuta el evento interno `ESTIMATE_WEIGHT`, enviando al servicio de IA los parámetros morfométricos ya convertidos a unidades métricas. El servicio de estimación de peso responde con un evento `{weight}`, que desencadena en el backend la ejecución del proceso `Classify`. En este proceso, el backend asigna la categoría Small, Medium o Large aplicando los umbrales establecidos, actualiza los contadores $N_k$, calcula la distribución porcentual $p_k$ y actualiza medias y biomasa acumulada. Una vez clasificado el pez, se genera un evento MQTT `classification/result` notificando la categoría que desencadena los eventos de control mecatrónico de compuertas, a la que vez se produce un evento de actualización hacia la interfaz que refresca en tiempo real el estado visual de los contadores.

El usuario puede intervenir en la sesión mediante tres eventos principales: `PAUSE`, `RESUME` y `STOP`. El evento `PAUSE` ordena la suspensión inmediata de la clasificación, enviando desde backend hacia MQTT un comando `{command:"pause"}` y congelando el consumo de nuevos mensajes sin perder contadores ni estadísticas. El evento `RESUME` revierte esta situación, reactivando el flujo de medidas con un comando `{command:"resume"}` y retornando al estado operativo activo. Por su parte, el evento `STOP` representa el cierre formal de la sesión: el backend ejecuta la desvinculación del tópico de visión, detiene cualquier procesamiento pendiente y calcula las estadísticas finales, generando un conjunto consolidado de métricas que incluye totales por categoría, distribuciones finales, medias de peso y biomasa estimada.

Una vez detenida la sesión, el evento `SAVE_SESSION` permite consolidar los resultados. El backend agrupa todos los datos en un bloque transaccional y ejecuta inserciones en la base de datos: un registro resumen de la sesión y un conjunto de documentos por cada medición procesada. 

```json
{
  "session_id": "<string: ISO8601 UTC timestamp | unique session identifier>",
  "measurements": {
    "fish_id": "<string: UUIDv4>",
    "timestamp": "<string: ISO8601 UTC timestamp>",
    "area_px": "<integer>",
    "length_px": "<integer>",
    "height_px": "<integer>",
    "area_mm": "<float>",
    "length_px": "<float>",
    "height_px": "<float>"
  }
}
```
Tras concluir la operación con éxito, se produce el evento `SAVE_CONFIRMED`, retornado a la interfaz para informar al usuario del guardado correcto.

De forma transversal al flujo operativo, el sistema implementa un mecanismo de supervisión basado en *heartbeats* periódicos intercambiados entre backend, visión y PLC. El backend publica periódicamente eventos de estado y espera los equivalentes de los módulos remotos. Si dichos heartbeats no llegan dentro del tiempo permitido, se genera automáticamente un evento `ConnectionLost`, que provoca una transición inmediata a un estado de seguridad (`error_paused`), detiene preventivamente la clasificación y notifica al usuario mediante un mensaje explícito. Este mecanismo asegura la robustez del sistema frente a desconexiones intermitentes y garantiza que cualquier pérdida de sincronización no comprometa ni la operación mecánica ni el bienestar de los peces.